<a href="https://colab.research.google.com/github/feranzie/Certification_projects/blob/main/Copy_of_Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-08-27 13:09:21--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2022-08-27 13:09:21 (10.4 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-08-27 13:09:21--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
with open(train_file_path) as fd:
    train_data = pd.read_csv(fd, delimiter="\t", quotechar='"',names=["class", "text"])
  
with open(test_file_path) as fd:
    test_data = pd.read_csv(fd, delimiter="\t", quotechar='"',names=["class", "text"])  

In [ ]:
train_data.head()

,class,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
train_label = np.array([0 if x=="ham" else 1 for x in train_data['class'].values.tolist()])
test_label = np.array([0 if x=="ham" else 1 for x in test_data['class'].values.tolist()])

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_data["text"].values, train_label)
)

In [ ]:
test_labelz =  test_data["class"].values
test_ds = tf.data.Dataset.from_tensor_slices(
    (test_data["text"].values, test_label)
)
test_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
def create_dict(list):
    word_count={}
    for unique in list:
      for word in unique.split():
        if word in word_count:
            word_count[word]+=1
        else:
            word_count[word]=1
    print(word_count)
    return word_count
word_count=create_dict(train_message)
len(word_count)

{'ahhhh...just': 1, 'woken': 2, 'up!had': 1, 'a': 1052, 'bad': 18, 'dream': 9, 'about': 122, 'u': 729, 'tho,so': 1, 'i': 1633, 'dont': 108, 'like': 182, 'right': 47, 'now': 181, ':)': 43, 'didnt': 20, 'know': 176, 'anything': 47, 'comedy': 3, 'night': 64, 'but': 325, 'guess': 26, 'im': 63, 'up': 199, 'for': 536, 'it.': 42, 'you': 1451, 'can': 295, 'never': 35, 'do': 290, 'nothing': 20, 'sound': 6, 'manky': 1, 'scouse': 1, 'boy': 17, 'steve,like!': 1, 'is': 643, 'travelling': 1, 'on': 400, 'da': 59, 'bus': 18, 'home.wot': 1, 'has': 95, 'inmind': 1, '4': 194, 'recreation': 1, 'dis': 21, 'eve?': 3, 'mum': 9, 'say': 65, 'we': 258, 'wan': 35, 'to': 1670, 'go': 213, 'then': 159, 'go...': 1, 'she': 108, 'shun': 1, 'bian': 1, 'watch': 25, 'glass': 1, 'exhibition...': 1, 'y': 27, 'lei...': 9, 'v': 27, 'lazy...': 1, 'got': 181, 'wat?': 4, 'dat': 26, 'day': 93, 'ü': 121, 'send': 139, 'me': 455, 'url': 1, 'cant': 48, 'work': 55, 'one...': 4, 'in': 651, 'xam': 1, 'hall': 1, 'asked': 17, 'girl': 18,

11330

In [ ]:
vec = TextVectorization(
    output_mode='int',
    max_tokens=11330,
    output_sequence_length=1000,
)

vec.adapt(train_ds.map(lambda text, label: text))

In [ ]:
vocab = np.array(vec.get_vocabulary())

In [ ]:
vocab

array(['', '[UNK]', 'to', ..., '01223585236', '0121', '0089my'],
      dtype='<U48')

In [ ]:
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, 1000)             0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 1000, 64)          523584    
                                                                 
 bidirectional_8 (Bidirectio  (None, 1000, 128)        66048     
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                      

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=10,
)


Epoch 1/10
131/131 [==============================] - 45s 202ms/step - loss: 0.5931 - accuracy: 0.8660 - val_loss: 0.4557 - val_accuracy: 0.8604
Epoch 2/10
131/131 [==============================] - 29s 220ms/step - loss: 0.3056 - accuracy: 0.8662 - val_loss: 0.1809 - val_accuracy: 0.8615
Epoch 3/10
131/131 [==============================] - 21s 161ms/step - loss: 0.1158 - accuracy: 0.9608 - val_loss: 0.0808 - val_accuracy: 0.9812
Epoch 4/10
131/131 [==============================] - 21s 161ms/step - loss: 0.0533 - accuracy: 0.9909 - val_loss: 0.0603 - val_accuracy: 0.9844
Epoch 5/10
131/131 [==============================] - 22s 170ms/step - loss: 0.0314 - accuracy: 0.9943 - val_loss: 0.0567 - val_accuracy: 0.9844
Epoch 6/10
131/131 [==============================] - 21s 161ms/step - loss: 0.0200 - accuracy: 0.9978 - val_loss: 0.0576 - val_accuracy: 0.9844
Epoch 7/10
131/131 [==============================] - 21s 162ms/step - loss: 0.0138 - accuracy: 0.9981 - val_loss: 0.0626 - val_ac

In [ ]:
def predict_message(pred_text):
    predict = model.predict([pred_text])
    prediction = predict[0][0]
    return [prediction, "ham" if prediction <0.5 else "spam"]


pred_text = "how are you doing today"
prediction = predict_message(pred_text)
print(prediction)

[-9.131605, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
